In [69]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [70]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [71]:
data= pd.read_csv('../data.csv')

In [72]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=5, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [76]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [77]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")



----------
best parameters:  {'criterion': 'entropy', 'max_depth': 13}
----------
[[11033  4326]
 [  571 14836]]
----------
              precision    recall  f1-score   support

           0       0.95      0.72      0.82     15359
           1       0.77      0.96      0.86     15407

    accuracy                           0.84     30766
   macro avg       0.86      0.84      0.84     30766
weighted avg       0.86      0.84      0.84     30766

----------
